# Notebook details

In [ ]:
def setup_notebook(fix_python_path=True, reduce_margins=True, plot_inline=True):
    if reduce_margins:
        # Reduce side margins of the notebook
        from IPython.core.display import display, HTML
        display(HTML("<style>.container { width:100% !important; }</style>"))

    if fix_python_path:
        # add egosocial to the python path
        import os, sys
        sys.path.extend([os.path.dirname(os.path.abspath('.'))])

    if plot_inline:
        # Plots inside cells
        %matplotlib inline
    
    global __file__
    __file__ = 'Notebook'

setup_notebook()

# Imports and Constants Definition

In [ ]:
# !/usr/bin/env python
# -*- coding: utf-8 -*-

import argparse
import json
import logging
import os

import IPython
import ipywidgets as widgets

import cv2

import egosocial
from egosocial import config
from egosocial.core.types import FaceClustering
from egosocial.utils.filesystem import check_directory 
from egosocial.utils.filesystem import create_directory
from egosocial.utils.filesystem import list_files_in_segment
from egosocial.utils.filesystem import list_segments
from egosocial.utils.logging import setup_logging
from egosocial.utils.parser import FACE_DETECTION
from egosocial.utils.parser import load_faces_from_file

DOMAINS = ['Attachent', 'Reciprocity', 'Mating', 'Heirarchical Power', 'Coalitional Group']
RELATIONS = [
    ['father-child', 'mother-child', 'grandpa-grandchild', 'grandma-grandchild'],
    ['friends', 'siblings', 'classmates'],
    ['lovers/spouses'],
    ['presenter-audience', 'teacher-student', 'trainer-trainee', 'leader-subordinate', 'customer-staff'],
    ['band members', 'dance team members', 'sport team members', 'colleages'],
]

def relation_to_domain(rel_label):
    for dom_idx, grouped_relations in enumerate(RELATIONS):
        for relation in grouped_relations:
            if rel_label == relation:
                return DOMAINS[dom_idx]
    
    for domain in DOMAINS:
        if domain in rel_label:
            return domain
    
    return rel_label

In [ ]:
if not os.path.isdir(egosocial.config.TMP_DIR):
    os.mkdir(egosocial.config.TMP_DIR)

setup_logging(egosocial.config.LOGGING_CONFIG,
              log_dir=egosocial.config.LOGS_DIR)

In [ ]:
#!pip install pandas
import pandas as pd

def load_labels(labels_dir, labels_file_name):
    segments = sorted(list_segments(labels_dir), key=int)
    labels_data = []
    
    for segment_id in segments:
        segm_labels_dir = os.path.join(labels_dir, segment_id)
        check_directory(segm_labels_dir, 'Labels')    
        
        labels_file = os.path.join(segm_labels_dir, labels_file_name)
        if not os.path.exists(labels_file):
            continue
        
        with open(labels_file, 'r') as json_file:
            labels_per_segment = json.load(json_file)
        
        for labels_group_id in sorted(labels_per_segment, key=lambda x : int(x['group_id'])):
            label_list, group_id = labels_group_id['labels'], int(labels_group_id['group_id'])
            if not label_list:
                label_list = ['unknown']

            for str_label in label_list:
                entry = int(segment_id), int(group_id), str_label, relation_to_domain(str_label)
                labels_data.append(entry)
    
    labels_df = pd.DataFrame(labels_data, columns=['segment_id', 'group_id', 'label', 'domain_label'])
    
    return labels_df

In [ ]:
base_labels_dir = '/media/emasa/OS/Users/Emanuel/Downloads/NO_SYNC/Social Segments/labels/'
labels_file_name = 'labels.json'

train_labels_dir = base_labels_dir + 'train'

train_labels_df = load_labels(train_labels_dir, labels_file_name)

In [ ]:
#train_labels_df.label.value_counts().plot(kind='pie', figsize=(10, 10))
train_labels_df.label.value_counts().plot(kind='barh', figsize=(10, 10))

In [ ]:
test_labels_dir = base_labels_dir + 'test'

test_labels_df = load_labels(test_labels_dir, labels_file_name)

In [ ]:
test_labels_df.label.value_counts().plot(kind='pie', figsize=(10, 10))
#test_labels_df.label.value_counts().plot(kind='barh', figsize=(10, 10))

In [ ]:
extended_labels_dir = base_labels_dir + 'extended'

extended_labels_df = load_labels(extended_labels_dir, labels_file_name)

In [ ]:
#extended_labels_df.label.value_counts().plot(kind='pie', figsize=(10, 10))
extended_labels_df.label.value_counts().plot(kind='barh', figsize=(10, 10))

In [ ]:
all_labels_df = pd.concat([train_labels_df, test_labels_df, extended_labels_df])

In [ ]:
all_labels_df.label.value_counts().plot(kind='barh', figsize=(10, 10))

In [ ]:
all_labels_df.domain_label.value_counts().plot(kind='barh', figsize=(10, 10))data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAqQAAAJCCAYAAAAWW9U2AAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1+/AAAADl0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uIDIuMS4yLCBodHRwOi8vbWF0cGxvdGxpYi5vcmcvNQv5yAAAIABJREFUeJzt3Xu0ZXdZ5+vvSwpIMBAIxBxEoVCu4VZggaKBwU0PAi0iaKA5SoAekT4Bmx6gRLtPi9BKGu2GRhBOVCDSCIiCJ00zUCRBoiChQu4hIELolhaQW+QaSHjPH3sWbDZ71yWV1Jtd+3nGqLHnmnOuOX/rl1nhw1xr7VR3BwAAptxgegAAAGxtghQAgFGCFACAUYIUAIBRghQAgFGCFACAUYIUAIBRghQAgFGCFACAUdumB8D+udWtbtXbt2+fHgYAwF6de+65n+7uY/a2nyDdZLZv355du3ZNDwMAYK+q6mP7sp+37AEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABi1bXoA7J9PfuTD+c8nPGp6GHv1rDe8ZXoIAMAm4Q4pAACjBCkAAKMEKQAAowQpAACjBCkAAKMEKQAAowQpAACjBCkAAKMEKQAAowQpAACjBCkAAKMEKQAAowQpAACjBCkAAKMEKQAAowQpAACjBCkAAKMEKQAAowTpGlXVVfXfVj3eVlX/VFVv2cvzdlTVI1Y9/smqOuW6HCsAwKFAkH6nLyW5e1UdsTz+sSQf34fn7UjyzSDt7jO6+9TrYHwAAIcUQbq+tyZ55LL8hCSv272hqu5XVe+pqvOq6t1VdeequlGS5yU5oarOr6oTqurEqnrp8pxXV9VLlv0/UlWPW9bfoKp+t6ouq6q3V9Vbd28DANgqBOn6Xp/k8VV1eJJ7Jnnvqm2XJXlAd987yX9I8pvd/bVl+Q3dvaO737DOMW+d5Pgkj0qy+87pTyfZnuS4JD+X5P7XwWsBALhe2zY9gOuj7r6wqrZn5e7oW9dsPirJ6VV1xySd5Ib7eNg/6+5vJLm0qo5d1h2f5I3L+k9U1VnrPbGqTkpyUpLc4iZHrLcLAMCm5Q7pxs5I8ttZ9Xb94vlJzuruuyf5F0kO38fjXblqufZnIN19Wnfv7O6d33XjG+3PUwEArvcE6cZemeTXu/uiNeuPyre+5HTiqvVfSHLT/TzH3yR57PJZ0mOTPOgajBMAYFMTpBvo7n/o7pess+mFSV5QVefl2z/ycFaS43Z/qWkfT/OnSf4hyaVJ/luS9ye54gCGDQCw6VR3T49hS6uqI7v7i1V1yyTnJPnR7v7ERvt/39E372f+2PEHb4DX0LPesMdf2woAbAFVdW5379zbfr7UNO8tVXXzJDdK8vw9xSgAwKFIkA7r7gdNjwEAYJLPkAIAMEqQAgAwSpACADBKkAIAMEqQAgAwSpACADBKkAIAMEqQAgAwSpACADBKkAIAMEqQAgAwSpACADBKkAIAMEqQAgAwSpACADBq2/QA2D/Hfv8d8qw3vGV6GAAA1xp3SAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYtW16AOyfT33sC3nZ086cHsYh4+RXPGR6CACw5blDCgDAKEEKAMAoQQoAwChBCgDAKEEKAMAoQQoAwChBCgDAKEEKAMAoQQoAwChBCgDAKEEKAMAoQQoAwChBCgDAKEEKAMAoQQoAwChBCgDAKEEKAMAoQQoAwChBCgDAqEM2SKvqp6qqq+ouy+PtVfUvV23fUVWPOIDjX15Vt7qWxvqr18ZxAAA2o0M2SJM8IclfLz+TZHuSf7lq+44k1zhIr2WCFADYsg7JIK2qI5Mcn+SpSR6/rD41yQOq6vyqek6S5yU5YXl8QlXdr6reU1XnVdW7q+rOy7EOq6rfrqqLq+rCqnrGqlM9o6reX1UXrboT+11V9cqqOmc51qOX9SdW1Zuq6m1V9XdV9cJl/alJjljG8dqDMkEAANcj26YHcB15dJK3dfeHquozVfWDSU5J8uzuflSSVNUnk+zs7qcvj2+W5AHdfVVVPSzJbyZ5bJKTsnJ3dcey7ehV5/l0d9+nqv7vJM9O8q+S/LskZ3b3U6rq5knOqaq/XPbfkeTeSa5M8sGq+p3uPqWqnt7dOzZ6MVV10jKO3OLI77425gcA4HrjkLxDmpW36V+/LL8+33rbfk+OSvLGqro4yYuS3G1Z/7Ak/293X5Uk3f3ZVc950/Lz3KxEa5L8eJJTqur8JO9McniS2y7b3tHdV3T3V5NcmuR2+/Jiuvu07t7Z3TuPPPzm+/IUAIBN45C7Q7rcwXxIkntUVSc5LEkn+R97eerzk5zV3Y+pqu1Zicm9uXL5eXW+NZeV5LHd/cE14/qhVfuvfQ4AwJZ1KN4hfVyS13T37bp7e3d/X5KPJvlGkpuu2u8Lax4fleTjy/KJq9a/PckvVNW25JvBuyd/npXPltay/733Ycxfr6ob7sN+AACHnEMxSJ+Q5M1r1v1pVr7cdHVVXVBV/zbJWUmO2/2lpiQvTPKCqjov337n8veT/M8kF1bVBfn2b+qv5/lJbrjsf8nyeG9OW/b3pSYAYMup7p4eA/vhtsfcuZ/z2JdPD+OQcfIrHjI9BAA4ZFXVud29c2/7HYp3SAEA2EQEKQAAowQpAACjBCkAAKMEKQAAowQpAACjBCkAAKMEKQAAowQpAACjBCkAAKMEKQAAowQpAACjBCkAAKMEKQAAowQpAACjBCkAAKMEKQAAo7ZND4D98923u2lOfsVDpocBAHCtcYcUAIBRghQAgFGCFACAUYIUAIBRghQAgFGCFACAUYIUAIBRghQAgFGCFACAUYIUAIBRghQAgFGCFACAUYIUAIBRghQAgFGCFACAUYIUAIBRghQAgFGCFACAUYIUAIBRghQAgFGCFACAUYIUAIBRghQAgFGCFACAUYIUAIBRghQAgFGCFACAUYIUAIBRghQAgFGCFACAUYIUAIBRghQAgFGCFACAUYIUAIBRghQAgFGCFACAUYIUAIBRghQAgFGCFACAUYIUAIBRghQAgFGCFACAUYIUAIBRghQAgFHbpgfA/vnqxZfkA3e56/QwOAjuetkHpocAAAeFO6QAAIwSpAAAjBKkAACMEqQAAIwSpAAAjBKkAACMEqQAAIwSpAAAjBKkAACMEqQAAIwSpAAAjBKkAACMEqQAAIwSpAAAjBKkAACMEqQAAIwSpAAAjBKkAACMEqRrVNWJVfXS6XEAAGwVghQAgFGHfJBW1faqunjV42dX1XOr6p1V9Z+q6pyq+lBVPWCd5z6yqt5TVbeqqldX1Uuq6t1V9ZGqetyyT1XVb1XVxVV1UVWdsKx/WVX95LL85qp65bL8lKr6jWVcH6iq36uqS6rqL6rqiIMzKwAA1x+HfJDuxbbuvl+SZyb5tdUbquoxSU5J8oju/vSy+tZJjk/yqCSnLut+OsmOJPdK8rAkv1VVt05ydpLdkXubJMctyw9I8q5l+Y5JXtbdd0vy+SSPvVZfHQDAJrDVg/RNy89zk2xftf4hSZ6T5JHd/blV6/+su7/R3ZcmOXZZd3yS13X31d39ySR/leS+WYK0qo5LcmmSTy6hev8k716e+9HuPn+DMXxTVZ1UVbuqatdnr77qmr9aAIDroa0QpFfl21/n4auWr1x+Xp1k26r1f5/kpknutOZYV65arj2dtLs/nuTmSR6elTuiZyf52SRf7O4vrHO8tWNYfazTuntnd+88+rB1dwEA2LS2QpB+Msl3V9Utq+rGWXm7fW8+lpW3z/+wqu62l33PTnJCVR1WVcckeWCSc5Ztf5uVjwPsDtJnLz8BAFgc8kHa3V9P8rysROLbk1y2j8+7LMkTk7yxqn5gD7u+OcmFSS5IcmaSX+7uTyzbzs7K51Q/nOT9SY6OIAUA+DbV3dNjYD/c/fAj+o3bt08Pg4Pgrpd9YHoIAHBAqurc7t65t/0O+TukAABcvwlSAABGCVIAAEYJUgAARglSAABGCVIAAEYJUgAARglSAABGCVIAAEYJUgAARglSAABGCVIAAEYJUgAARglSAABGCVIAAEYJUgAARglSAABGbZseAPvn8LvfLXfdtWt6GAAA1xp3SAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABi1bXoA7J9LPnNJ7nH6PaaHwUFw0ZMumh4CABwU7pACADBKkAIAMEqQAgAwSpACADBKkAIAMEqQAgAwSpACADBKkAIAMEqQAgAwSpACADBKkAIAMEqQAgAwSpACADBKkAIAMEqQAgAwSpACADBKkAIAMEqQAgAwaq9BWlVfXPP4xKp66V6e85NVdcqBDm6DY7+zqnaus35nVb3kWj7mO6vqg1V1QVX9TVXd+ZocHwCAjV0nd0i7+4zuPnXt+qrati/P39f91pxzV3f/4v4+bx88sbvvleT0JL91HRz/21yT1w4AsJkdUJBW1TFV9adV9b7lz48u6795F7WqXl1Vr6iq9yZ5YVXdr6reU1XnVdW7d991XJ5zRlWdmeQdy7rnVNVFyx3K1YH7M1V1TlV9qKoesOz7oKp6y7J8ZFW9annuhVX12GX9y6tqV1VdUlW/vp8v911J7rAc56HL+C+qqldW1Y2r6r5V9aZl+6Or6itVdaOqOryqPrKs/4GqeltVnVtVZ1fVXdabo/3/JwEAsHnty924I6rq/FWPj05yxrL8X5O8qLv/uqpum+TPk9x1nWN8b5If6e6rq+pmSR7Q3VdV1cOS/GaSxy773SfJPbv7s1X1E0keneSHuvvLVXX06nF39/2q6hFJfi3Jw9ac7/9JckV33yNJquoWy/p/txz7sCTvqKp7dveF+zAHSfIvklxUVYcneXWSh3b3h6rqD5P86yQvTbJj2fcBSS5Oct+szPF7l/WnJXlad/9dVf1Qkt9N8pC1c7T2xFV1UpKTkuSGt7zhPg4XAGBz2Jcg/Up37w6tVNWJSXZ/3vJhSY6rqt2bb1ZVR65zjDeuCq2jkpxeVXdM0klWF9bbu/uzq479qu7+cpKsWp8kb1p+nptk+zrne1iSx+9+0N2fWxZ/dom7bUluneS4JHsL0tdW1VeSXJ7kGUnunOSj3f2hZfvpSU7u7hdX1d9X1V2T3C/Jf0nywCSHJTl7mZcfSfLGVfN141XneeN6MbqM/7SsxGyOuP0RvZfxAgBsKgf6ecUbJPnh7v7q6pWrgmu3L61afn6Ss7r7MVW1Pck7N9hvT65cfl6dfXwNVXX7JM9Oct/u/lxVvTrJ4fvw1Cd2965Vxzl6D/u+K8lPJPl6kr/Myp3Uw5L8Ulbm6vOr436NfX3tAACHlAP9UtNfZOWuYZKkqjaKrdWOSvLxZfnEPez39iRPrqqbLMfeUwiu99yTV43rFklulpXou6Kqjs1KOF4TH0yyvarusDz+uSR/tSyfneSZSd7T3f+U5JZZuaN6cXf/c5KPVtXPLGOqqrrXNRwDAMAh40CD9BeT7Fy+OHRpkqftw3NemOQFVXVe9nB3s7vflpXPqu5aPsP67P0Y139McouquriqLkjy4O6+IMl5SS5L8kdJ/mY/jrd6XF9N8uSsvPV+UZJvJHnFsvm9SY7Nyp3SZOXjABd19+632Z+Y5KnLmC7JymdkAQC2tPpWK7EZHHH7I/oOz73D3ndk07voSRdNDwEADkhVndvd3/G73tfyX2oCAGCUIAUAYJQgBQBglCAFAGCUIAUAYJQgBQBglCAFAGCUIAUAYJQgBQBglCAFAGCUIAUAYJQgBQBglCAFAGCUIAUAYJQgBQBglCAFAGCUIAUAYNS26QGwf+52y7tl15N2TQ8DAOBa4w4pAACjBCkAAKMEKQAAowQpAACjBCkAAKMEKQAAowQpAACjBCkAAKMEKQAAowQpAACjBCkAAKMEKQAAowQpAACjBCkAAKMEKQAAowQpAACjBCkAAKMEKQAAowQpAACjBCkAAKMEKQAAowQpAACjBCkAAKMEKQAAowQpAACjBCkAAKMEKQAAowQpAACjBCkAAKMEKQAAowQpAACjBCkAAKMEKQAAowQpAACjBCkAAKMEKQAAowQpAACjBCkAAKMEKQAAowQpAACjBCkAAKMEKQAAowQpAACjBCkAAKO2TQ+A/fS/z0uee9T0KGDzeu4V0yMAYA13SAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABh1QEFaVf9HVb2+qv6+qs6tqrdW1Z2u4bG+uPz8nqr6k2V5R1U9YtU+P1lVpxzImPd07nXWH1tVf1RVH1le33uq6jHX9vkBALayaxykVVVJ3pzknd39A939g0l+JcmxBzKg7v7f3f245eGOJI9Yte2M7j71QI6/r5bX92dJ3tXd37+8vscn+d519t12MMYEAHAoOpA7pA9O8vXufsXuFd19QXefXSt+q6ourqqLquqEJKmqI6vqHVX1/mX9o9cetKq2L8+7UZLnJTmhqs6vqhOq6sSqeumq/c6sqguXY952Wf/qqnpJVb17ubP5uH099xoPSfK1Na/vY939O8vxTqyqM6rqzCTv2MNrflBVvWXV63tpVZ24LF9eVS9c9j+nqu6w3/8UAAA2uQO5s3f3JOdusO2ns3J3815JbpXkfVX1riT/lOQx3f3PVXWrJH9bVWd0d689QHd/rar+Q5Kd3f30ZCUCV+3yO0lO7+7Tq+opSV6S5KeWbbdOcnySuyQ5I8mfJPnqvp57cbck79/LHNwnyT27+7NV9dgNXvPeXNHd96iqn0/y4iSP2ofnAAAcMq6rLzUdn+R13X11d38yyV8luW+SSvKbVXVhkr9Mcptc87f475/kj5bl1yzn3O3Puvsb3X3pquMf0Lmr6mVVdUFVvW/V6rd392eX5Y1e8968btXP+29w7pOqaldV7fqnL2/UzwAAm9OBBOklSX5wP5/zxCTHJPnB7t6R5JNJDj+AMWzkylXLdQ3PfUlW7oAmSbr75CQPXY6x25f2YSxX5dvnee05e4Plb63sPq27d3b3zmNuUuvtAgCwaR1IkJ6Z5MZVddLuFVV1z6p6QJKzs/LZz8Oq6pgkD0xyTpKjknyqu79eVQ9Ocru9nOMLSW66wbZ3Z+VLRslKbJ69l2Pt77nPTHJ4Vf3rVetusof9N3rNH0tyXFXduKpunpWoXe2EVT/fs5cxAQAccq7xZ0i7u5dfgfTiqnpOVj6jeXmSZyb566y8/XxBVu76/XJ3f6KqXpvkv1fVRUl2JblsL6c5K8kpVXV+khes2faMJK+qql/KymdTn7yXY+3XuZfX91NJXlRVv7yc40tJnrPBU96cdV5zklTVHye5OMlHk5y35nm3WD5GcGWSJ+zlNQAAHHJq4+/0cF2rqsuz8qWtT+/rc3Z+z2G966Qjr7tBwaHuuVdMjwBgy6iqc7t75972819qAgBglF/oPqi7t0+PAQBgmjukAACMEqQAAIwSpAAAjBKkAACMEqQAAIwSpAAAjBKkAACMEqQAAIwSpAAAjBKkAACMEqQAAIwSpAAAjBKkAACMEqQAAIwSpAAAjBKkAACM2jY9APbT99w7ee6u6VEAAFxr3CEFAGCUIAUAYJQgBQBglCAFAGCUIAUAYJQgBQBglCAFAGCUIAUAYJQgBQBglCAFAGCUIAUAYJQgBQBglCAFAGCUIAUAYJQgBQBglCAFAGCUIAUAYJQgBQBglCAFAGCUIAUAYJQgBQBglCAFAGCUIAUAYJQgBQBglCAFAGCUIAUAYJQgBQBglCAFAGCUIAUAYJQgBQBglCAFAGCUIAUAYJQgBQBglCAFAGCUIAUAYJQgBQBglCAFAGCUIAUAYJQgBQBglCAFAGCUIAUAYJQgBQBglCAFAGCUIAUAYNS26QGwfy76+BXZfsr/mB4GALBJXX7qI6eH8B3cIQUAYJQgBQBglCAFAGCUIAUAYJQgBQBglCAFAGCUIAUAYJQgBQBglCAFAGCUIAUAYJQgBQBglCAFAGCUIAUAYJQgBQBglCAFAGCUIAUAYJQgBQBglCAFAGDUpg3Sqrq6qs6vqour6r9X1c2v4XGeVlU/f22Pb805fr+qjluWf/W6PBcAwGazaYM0yVe6e0d33z3JZ5OcfE0O0t2v6O4/3Nf9q2rbNTjHv+ruS5eHghQAYJXNHKSrvSfJbXY/qKpfqqr3VdWFVfXrq9b//LLugqp6zbLuuVX17GX5nVX1X1fdeb3fqn1eU1V/k+Q1VXV4Vb2qqi6qqvOq6sHLfodV1W8vz72wqp6x6rg7q+rUJEcsx39tVT2vqp65any/UVX/5iDMFwDA9cZ+3+27vqmqw5I8NMkfLI9/PMkdk9wvSSU5o6oemOQzSf59kh/p7k9X1dEbHPIm3b1jec4rk9x9WX9ckuO7+ytV9awk3d33qKq7JPmLqrpTkicn2Z5kR3dftfYc3X1KVT29u3csY92e5E1JXlxVN0jy+GXcAABbxmYO0iOq6vys3Bn9QJK3L+t/fPlz3vL4yKwE6r2SvLG7P50k3f3ZDY77umX7u6rqZqs+m3pGd39lWT4+ye8s+11WVR9LcqckD0vyiu6+ai/nyLL98qr6TFXdO8mxSc7r7s+s3a+qTkpyUpIcdrNj9nRIAIBNZzO/Zf+V5U7j7bJyJ3T3Z0gryQuWz5fu6O47dPcf7Mdxe4PHXzqw4W7o95OcmJW7q69cd0Ddp3X3zu7eedhNjrqOhgEAMGMzB2mSpLu/nOQXkzxr+cLRnyd5SlUdmSRVdZuq+u4kZyb5maq65bJ+o7fsT1i2H5/kiu6+Yp19zk7yxGW/OyW5bZIPZuUu7S/s/uLTBuf4elXdcNXjNyd5eJL7LmMHANhSNvNb9t/U3edV1YVJntDdr6mquyZ5T1UlyReT/F/dfUlV/UaSv6qqq7Pylv6J6xzuq1V1XpIbJnnKBqf83SQvr6qLklyV5MTuvrKqfj8rb91fWFVfT/J7SV665rmnLdvf391P7O6vVdVZST7f3VcfwDQAAGxK1b32Heqtq6remeTZ3b3rIJ7zBknen+Rnuvvv9rb/jW99x771k1583Q8MADgkXX7qIw/auarq3O7eubf9Nv1b9pvZ8svyP5zkHfsSowAAh6JD4i37a0t3P+ggn+/SJN9/MM8JAHB94w4pAACjBCkAAKMEKQAAowQpAACjBCkAAKMEKQAAowQpAACjBCkAAKMEKQAAowQpAACjBCkAAKMEKQAAowQpAACjBCkAAKMEKQAAo7ZND4D9c4/bHJVdpz5yehgAANcad0gBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABglSAEAGCVIAQAYJUgBABhV3T09BvZDVX0hyQenx3E9dKskn54exPWQeVmfeVmfeVmfeVmfedmYufmW23X3MXvbadvBGAnXqg92987pQVzfVNUu8/KdzMv6zMv6zMv6zMv6zMvGzM3+85Y9AACjBCkAAKME6eZz2vQArqfMy/rMy/rMy/rMy/rMy/rMy8bMzX7ypSYAAEa5QwoAwChBuklU1cOr6oNV9eGqOmV6PJOq6vKquqiqzq+qXcu6o6vq7VX1d8vPW0yP82CoqldW1aeq6uJV69adi1rxkuUaurCq7jM38uvWBvPy3Kr6+HLdnF9Vj1i17VeWeflgVf2fM6O+7lXV91XVWVV1aVVdUlX/Zlm/pa+ZPczLlr5mqurwqjqnqi5Y5uXXl/W3r6r3Lq//DVV1o2X9jZfHH162b58c/3VlD/Py6qr66KrrZceyfkv8PTpQgnQTqKrDkrwsyU8kOS7JE6rquNlRjXtwd+9Y9Ws1Tknyju6+Y5J3LI+3glcnefiadRvNxU8kuePy56QkLz9IY5zw6nznvCTJi5brZkd3vzVJlr9Lj09yt+U5v7v8nTsUXZXkWd19XJIfTnLy8vq3+jWz0bwkW/uauTLJQ7r7Xkl2JHl4Vf1wkv+UlXm5Q5LPJXnqsv9Tk3xuWf+iZb9D0UbzkiS/tOp6OX9Zt1X+Hh0QQbo53C/Jh7v7I939tSSvT/Lo4TFd3zw6yenL8ulJfmpwLAdNd78ryWfXrN5oLh6d5A97xd8muXlV3frgjPTg2mBeNvLoJK/v7iu7+6NJPpyVv3OHnO7+x+5+/7L8hSQfSHKbbPFrZg/zspEtcc0s/9y/uDy84fKnkzwkyZ8s69deL7uvoz9J8tCqqoM03INmD/OykS3x9+hACdLN4TZJ/teqx/+QPf/L8lDXSf6iqs6tqpOWdcd29z8uy59IcuzM0K4XNpoL11Hy9OUts1eu+ljHlpyX5e3Ueyd5b1wz37RmXpItfs1U1WFVdX6STyV5e5K/T/L57r5q2WX1a//mvCzbr0hyy4M74oNj7bwY8zi7AAACtklEQVR09+7r5TeW6+VFVXXjZd2WuV4OhCBlMzq+u++TlbdBTq6qB67e2Cu/OsKvj4i5WOPlSX4gK2+x/WOS/zw7nDlVdWSSP03yzO7+59XbtvI1s868bPlrpruv7u4dSb43K3eB7zI8pOuFtfNSVXdP8itZmZ/7Jjk6yXMGh7jpCNLN4eNJvm/V4+9d1m1J3f3x5eenkrw5K/+S/OTut0CWn5+aG+G4jeZiS19H3f3J5X9EvpHk9/Ktt1i31LxU1Q2zEl2v7e43Lau3/DWz3ry4Zr6luz+f5Kwk98/KW867/9Pjq1/7N+dl2X5Uks8c5KEeVKvm5eHLRz+6u69M8qps4evlmhCkm8P7ktxx+WbjjbLyYfozhsc0oqq+q6puuns5yY8nuTgr8/GkZbcnJfn/ZkZ4vbDRXJyR5OeXb3z+cJIrVr1Ne8hb85mtx2TluklW5uXxyzeEb5+VLx6cc7DHdzAsn+f7gyQf6O7/smrTlr5mNpqXrX7NVNUxVXXzZfmIJD+Wlc/XnpXkcctua6+X3dfR45Kc2YfgLzvfYF4uW/V/6iorn6tdfb0c8n+PDtS2ve/CtO6+qqqenuTPkxyW5JXdfcnwsKYcm+TNy+fktyX5o+5+W1W9L8kfV9VTk3wsyc8OjvGgqarXJXlQkltV1T8k+bUkp2b9uXhrkkdk5QsYX07y5IM+4INkg3l50PJrWDrJ5Ul+IUm6+5Kq+uMkl2bl29Ynd/fVE+M+CH40yc8luWj5/FuS/GpcMxvNyxO2+DVz6ySnL79B4AZJ/ri731JVlyZ5fVX9xyTnZSXms/x8TVV9OCtfKnz8xKAPgo3m5cyqOiZJJTk/ydOW/bfK36MD4r/UBADAKG/ZAwAwSpACADBKkAIAMEqQAgAwSpACADBKkAIAMEqQAgAwSpACADDq/wcpgCdb1iyG4wAAAABJRU5ErkJggg==

In [ ]:
relations = all_labels_df.label
filtered_labels = relations[relations != 'unknown']
counts = filtered_labels.value_counts()
counts[counts > 10]

In [ ]:
domains = all_labels_df.domain_label
filtered_labels = domains[domains != 'unknown']
counts = filtered_labels.value_counts()
counts[counts > 10]